In [2]:
import numpy as np
import pandas as pd
from multiprocessing import Pool
import matplotlib.pyplot as plt
import matplotlib.gridspec as grid_spec
from tqdm.auto import tqdm
from math import ceil
import itertools
import h5py
import io
import robustbench

In [3]:
dataset_path = "/data/output/20220226_robustness/dataset.h5"
df_meta = pd.read_hdf(dataset_path, "meta")
df_meta["filter_ids"] = df_meta["filter_ids"].apply(lambda s: np.arange(int(s.split(":")[0]), 1 + int(s.split(":")[1])))
df_meta.Robust = df_meta.Robust.apply(bool)

In [4]:
df_pairings = pd.read_csv("robust_vs_normal_pairings.csv", index_col=0)

In [5]:
df_pairings.Robust.nunique()

71

In [6]:
df_pairings.Arch.nunique()

13

In [7]:
df_pairings.Dataset.nunique()

3

In [5]:
all_used_models = set(list(df_pairings.Robust.unique()) + list(df_pairings.Normal.unique()))

In [9]:
df_meta[df_meta.Name.isin(df_pairings.Robust)].filter_ids.apply(len).sum()

615863744

In [18]:
df_meta.groupby(["Robust", "Training-Dataset", "Backbone",]).Name.nunique()

Robust  Training-Dataset  Backbone               
False   cifar10           PreActResNet-18             3
                          ResNet-18                   2
                          ResNet-50                   2
                          WideResNet-106-16           1
                          WideResNet-28-10            3
                          WideResNet-28-10-PSSiLU     1
                          WideResNet-28-4             1
                          WideResNet-34-10            7
                          WideResNet-34-15            1
                          WideResNet-34-20            5
                          WideResNet-34-R             1
                          WideResNet-70-16            2
        cifar100          PreActResNet-18             3
                          WideResNet-28-10            2
                          WideResNet-34-10            4
                          WideResNet-34-20            1
                          WideResNet-70-16            

In [28]:
df_meta.groupby(["Robust", "Training-Dataset"]).Name.nunique()

Robust  Training-Dataset
False   cifar10             29
        cifar100            11
        imagenet             6
True    cifar10             49
        cifar100            17
        imagenet             5
Name: Name, dtype: int64

In [14]:
from robustbench.model_zoo import model_dicts as all_models
from robustbench.model_zoo.enums import BenchmarkDataset, ThreatModel
import inspect
from collections import defaultdict

paper_dict = defaultdict(lambda: defaultdict(list))
for dataset in BenchmarkDataset:
    for paper, info_dict in all_models[dataset][ThreatModel.Linf].items():
        paper_dict[dataset.value][inspect.getsource(info_dict["model"])].append(paper)

In [15]:
for dataset in BenchmarkDataset:
    for paper, info_dict in all_models[dataset][ThreatModel.Linf].items():
        if not paper in df_meta.Network.values:
            print(paper, dataset, "is missing")

Standard BenchmarkDataset.cifar_10 is missing
Kang2021Stable BenchmarkDataset.cifar_10 is missing
Standard_R50 BenchmarkDataset.imagenet is missing


In [19]:
pairings = []
for dataset in BenchmarkDataset:
    for paper, info_dict in all_models[dataset][ThreatModel.Linf].items():
        robust_model = df_meta[(df_meta.Network == paper) & (df_meta["Training-Dataset"] == dataset.value) & (df_meta.Robust)].Name.unique()
        
        if len(robust_model) < 1:
            print(paper, dataset.value)
            continue
        
        for pairs in paper_dict[dataset.value].values():
            if paper in pairs:
                counter_parts = pairs
        
        backbone = df_meta[(df_meta.Network.isin(counter_parts)) & (df_meta["Training-Dataset"] == dataset.value)].Backbone.unique()
                
        if dataset.value != "imagenet":
            normal_model = df_meta[(df_meta.Network.isin(counter_parts)) & (df_meta["Training-Dataset"] == dataset.value) & (~df_meta.Robust)].Name.unique()
        else:  # our imagenet models didnt train well, so let's use the ones from timm
            normal_model = df_meta[df_meta.Name.str.contains("timm_") & (df_meta.Backbone == backbone[0])].Name.unique()
        
       
        pairings.append((robust_model[0], normal_model[0], backbone[0], dataset.value))
        
df_pairings = pd.DataFrame(pairings, columns=["Robust", "Normal", "Arch", "Dataset"])
df_pairings

Standard cifar10
Kang2021Stable cifar10
Standard_R50 imagenet


,Robust,Normal,Arch,Dataset
0,robustbench_Andriushchenko2020Understanding_ci...,hso_normal_training_robustbench_Andriushchenko...,PreActResNet-18,cifar10
1,robustbench_Carmon2019Unlabeled_cifar10_linf_13,hso_normal_training_robustbench_Carmon2019Unla...,WideResNet-28-10,cifar10
2,robustbench_Sehwag2020Hydra_cifar10_linf_13,hso_normal_training_robustbench_Carmon2019Unla...,WideResNet-28-10,cifar10
3,robustbench_Wang2020Improving_cifar10_linf_13,hso_normal_training_robustbench_Carmon2019Unla...,WideResNet-28-10,cifar10
4,robustbench_Hendrycks2019Using_cifar10_linf_13,hso_normal_training_robustbench_Hendrycks2019U...,WideResNet-28-10,cifar10
...,...,...,...,...
66,robustbench_Wong2020Fast_imagenet_linf_13,timm_resnet50_imagenet1k_13,ResNet-50,imagenet
67,robustbench_Engstrom2019Robustness_imagenet_li...,timm_resnet50_imagenet1k_13,ResNet-50,imagenet
68,robustbench_Salman2020Do_R50_imagenet_linf_13,timm_resnet50_imagenet1k_13,ResNet-50,imagenet
69,robustbench_Salman2020Do_R18_imagenet_linf_13,timm_resnet18_imagenet1k_13,ResNet-18,imagenet


## Number of ronbust filters per dataset

In [13]:
df_meta[df_meta.Name.isin(df_pairings.Robust)].groupby("Training-Dataset").apply(lambda x: (x["in_channels"] * x["out_channels"]).sum())

Training-Dataset
cifar10     447825488
cifar100    158015344
imagenet     10022912
dtype: int64

In [29]:
df_meta[df_meta.Name.isin(df_pairings.Robust)].groupby(["Training-Dataset", "Name"]).apply(lambda x: (x["in_channels"] * x["out_channels"]).sum()).to_frame().reset_index().groupby("Training-Dataset").mean()

,0
Training-Dataset,
cifar10,9.139296e+06
cifar100,9.295020e+06
imagenet,2.004582e+06


In [15]:
df_pairings["Paper"] =  df_pairings.Robust.apply(lambda name: df_meta[df_meta.Name == name].Network.values[0])
df_pairings["Adv. Trained Clean Acc"] =  df_pairings.Robust.apply(lambda name: df_meta[df_meta.Name == name]["Clean Accuracy"].values[0])
df_pairings["Adv. Trained Robust Acc"] = df_pairings.Robust.apply(lambda name: df_meta[df_meta.Name == name]["Robust Accuracy"].values[0])
df_pairings["Norm. Trained Clean Acc"] = df_pairings.Normal.apply(lambda name: df_meta[df_meta.Name == name]["Clean Accuracy"].values[0])
df_pairings["Norm. Trained Robust Acc"] = df_pairings.Normal.apply(lambda name: df_meta[df_meta.Name == name]["Robust Accuracy"].values[0])

In [31]:
print(df_pairings.groupby("Dataset").agg({"Norm. Trained Clean Acc": "mean", "Adv. Trained Clean Acc": "mean", "Adv. Trained Robust Acc": "mean"}).to_latex(float_format="%.2f"))

\begin{tabular}{lrrr}
\toprule
{} &  Norm. Trained Clean Acc &  Adv. Trained Clean Acc &  Adv. Trained Robust Acc \\
Dataset  &                          &                         &                          \\
\midrule
cifar10  &                    92.17 &                   86.92 &                    56.71 \\
cifar100 &                    72.68 &                   62.24 &                    29.03 \\
imagenet &                    78.46 &                   60.72 &                    30.78 \\
\bottomrule
\end{tabular}



## Export

In [ ]:
df_pairings.to_csv("robust_vs_normal_pairings.csv")

In [22]:
print(df_pairings[["Paper", "Dataset", "Arch", "Adv. Trained Clean Acc", "Adv. Trained Robust Acc", "Norm. Trained Clean Acc", "Norm. Trained Robust Acc"]].to_latex(index=False, longtable=True))

\begin{longtable}{lllrrrr}
\toprule
                               Paper &  Dataset &                    Arch &  Adv. Trained Clean Acc &  Adv. Trained Robust Acc &  Norm. Trained Clean Acc &  Norm. Trained Robust Acc \\
\midrule
\endfirsthead

\toprule
                               Paper &  Dataset &                    Arch &  Adv. Trained Clean Acc &  Adv. Trained Robust Acc &  Norm. Trained Clean Acc &  Norm. Trained Robust Acc \\
\midrule
\endhead
\midrule
\multicolumn{7}{r}{{Continued on next page}} \\
\midrule
\endfoot

\bottomrule
\endlastfoot
     Andriushchenko2020Understanding &  cifar10 &         PreActResNet-18 &                   79.84 &                    43.93 &                    94.51 &                       0.0 \\
                 Carmon2019Unlabeled &  cifar10 &        WideResNet-28-10 &                   89.69 &                    59.53 &                    95.10 &                       0.0 \\
                     Sehwag2020Hydra &  cifar10 &        WideResNet-28-1